# Resource Catalog

This notebook demostrates usage of the EODH resource catalog API using `pyeodh`

First we need to create an instance of the `Client`, which is our entrypoint to EODH APIs.

In [1]:
import pyeodh

client = pyeodh.Client(base_url="https://test.eodatahub.org.uk") # Optionally specify a different server

Fetch the resource catalog object.

In [2]:
# GET /stac-fastapi/catalogs
catalogs = client.get_resource_catalogs()
rc = client.get_resource_catalog("stac-fastapi")

DEBUG: _request_json_raw received {'self': <pyeodh.client.Client object at 0x7fa4502b1190>, 'method': 'GET', 'url': '/stac-fastapi/catalogs', 'headers': None, 'params': None, 'data': None, 'encode': <function _encode_json at 0x7fa45004ea20>}
DEBUG: Received not absolute url: /stac-fastapi/catalogs
DEBUG: Created url from base: https://test.eodatahub.org.uk/stac-fastapi/catalogs
DEBUG: Making request: GET https://test.eodatahub.org.uk/stac-fastapi/catalogs
       headers: {}
       params: None
       body: None
DEBUG: Received response 200
       headers: {'Content-Type': 'application/json', 'Content-Length': '1151', 'Connection': 'keep-alive', 'Date': 'Fri, 17 May 2024 11:28:39 GMT', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 ed5d8b8e3a8c20eaabbb29c087f04c66.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'VIE50-C2', 'X-Amz-Cf-Id': 'lZOsJUgy1GdmNUv-XgpHjoDtaqZ7i

All attributes are mapped to properties, e.g.

In [4]:
for cat in catalogs:
    print("id: ", cat.id)
    print("title: ", cat.title)
    print("description: ", cat.description)
    print("")

id:  default-catalog
title:  
description:  Root catalog

id:  demo-workflows
title:  Supported Datasets
description:  Catalogue containing supported datasets

id:  earth-search-aws
title:  Earth Search by Element 84
description:  A STAC API of public datasets on AWS

id:  stac-fastapi
title:  Supported Datasets
description:  Catalogue containing supported datasets

id:  test-catalog
title:  
description:  Root catalog



API endpoints are wrapped in methods and are structured into classes following the same logic as the API. E.g. to fetch a collection item, I first need to get the collection from the resource catalog.

In [5]:
# GET /stac-fastapi/collections/{collection_id}/items/{item_id}
cmip6 = rc.get_collection("cmip6")
item = cmip6.get_item("CMIP6.ScenarioMIP.THU.CIESM.ssp585.r1i1p1f1.Amon.rsus.gr.v20200806")
print(item.id)

DEBUG: _request_json_raw received {'self': <pyeodh.client.Client object at 0x7fa4502b1190>, 'method': 'GET', 'url': 'https://test.eodatahub.org.uk/stac-fastapi/catalogs/stac-fastapi/collections/cmip6', 'headers': None, 'params': None, 'data': None, 'encode': <function _encode_json at 0x7fa45004ea20>}
DEBUG: Making request: GET https://test.eodatahub.org.uk/stac-fastapi/catalogs/stac-fastapi/collections/cmip6
       headers: {}
       params: None
       body: None
DEBUG: Received response 200
       headers: {'Content-Type': 'application/json', 'Content-Length': '3395', 'Connection': 'keep-alive', 'Date': 'Fri, 17 May 2024 11:29:25 GMT', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 ed5d8b8e3a8c20eaabbb29c087f04c66.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'VIE50-C2', 'X-Amz-Cf-Id': 'ipHZ3qoNepKhQzEf1cnZgGtRFcXZNWDf_zbuFJ9blfLkpezOUMzcmA=='}
       content: {"

HTTPError: 405 Client Error: Method Not Allowed for url: https://test.eodatahub.org.uk/stac-fastapi/catalogs/stac-fastapi/collections/cmip6/items/CMIP6.ScenarioMIP.THU.CIESM.ssp585.r1i1p1f1.Amon.rsus.gr.v20200806

Some API responses are paginated (e.g. collection items), and you can simply iterate over them.

In [5]:
# GET /stac-fastapi/collections/cmip6/items
items = cmip6.get_items()
for item in items:
    print(item.id)

CMIP6.ScenarioMIP.THU.CIESM.ssp585.r1i1p1f1.Amon.rsus.gr.v20200806
CMIP6.ScenarioMIP.THU.CIESM.ssp585.r1i1p1f1.Amon.rlus.gr.v20200806
CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp126.r1i1p1f1.day.uas.gn.v20210318
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.day.pr.gn.v20210317
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp126.r1i1p1f1.day.pr.gn.v20210317
CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.r1i1p1f1.day.rsds.gn.v20210318
CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.r1i1p1f1.day.hurs.gn.v20210318
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.day.tas.gn.v20210317
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.day.psl.gn.v20210317
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp126.r1i1p1f1.day.tasmin.gn.v20210317


Attempting to create a collection with id that already exists will result in 409 error code. To see the example in action delete the test collection first by running the cell below.

Delete a colletion

In [6]:
rc.get_collection("test1").delete()

Create new collection example

In [7]:
test1 = rc.create_collection(id="test1", title="Test", description="Test collection")
print(test1.description)

Test collection


Update a collection

In [8]:
test1.update(description="Different description")
print(test1.description)

Different description


Create an item

In [9]:
testitem1 = test1.create_item(id="test1.testitem1")
print(f"Created {testitem1.id} in collection {testitem1.collection}")

Created test1.testitem1 in collection test1


Update an item

In [10]:
testitem1.update(properties={"foo": "bar"})
print(testitem1.properties)

{'foo': 'bar', 'updated': '2024-04-30T14:08:44.187241Z', 'created': '2024-04-30T14:08:44.187241Z'}


Delete an item

In [11]:
testitem1.delete()

Find out more information about the resource catalog.

In [12]:
print(f"Livecheck: PING-{rc.ping()}")
print("\nAPI conforms to:", *rc.get_conformance(), sep="\n")

Livecheck: PING-PONG



API conforms to:
http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30
http://www.opengis.net/spec/cql2/1.0/conf/advanced-comparison-operators
http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter
http://www.opengis.net/spec/cql2/1.0/conf/cql2-text
http://www.opengis.net/spec/cql2/1.0/conf/basic-cql2
https://api.stacspec.org/v1.0.0-rc.2/item-search#filter
http://www.opengis.net/spec/cql2/1.0/conf/cql2-json
http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson
https://api.stacspec.org/v1.0.0/collections
https://api.stacspec.org/v1.0.0/ogcapi-features
https://api.stacspec.org/v1.0.0-rc.2/item-search#context
https://api.stacspec.org/v1.0.0/item-search#sort
http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter
https://api.stacspec.org/v1.0.0/item-search#fields
https://api.stacspec.org/v1.0.0-rc.3/ogcapi-features/extensions/transaction
https://api.stacspec.org/v1.0.0/core
https://api.stacspec.org/v1.0.0/item-search#query
http://www.opengis.net/spec/

Search the catalog

In [13]:
for result in rc.search(collections=['cmip6']):
    print(result.id)

CMIP6.ScenarioMIP.THU.CIESM.ssp585.r1i1p1f1.Amon.rsus.gr.v20200806
CMIP6.ScenarioMIP.THU.CIESM.ssp585.r1i1p1f1.Amon.rlus.gr.v20200806
CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp126.r1i1p1f1.day.uas.gn.v20210318
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.day.pr.gn.v20210317
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp126.r1i1p1f1.day.pr.gn.v20210317
CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.r1i1p1f1.day.rsds.gn.v20210318
CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.r1i1p1f1.day.hurs.gn.v20210318
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.day.tas.gn.v20210317
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.day.psl.gn.v20210317
CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp126.r1i1p1f1.day.tasmin.gn.v20210317
